In [1]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os
import time

# Import API key
from config import weather_api_key
from config import g_key


In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

weather_df = pd.read_csv("cities.csv")
weather_df

,Unnamed: 0,city,country,latitude,longitude,date,temperature,humidity,cloudiness,windspeed
0,0,La Rioja,ES,42.2500,-2.5000,NaN,9.16,29,5,1.81
1,1,Codrington,AU,-38.2667,141.9667,NaN,8.12,78,3,3.43
2,2,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,NaN,17.28,69,4,1.69
3,3,Bathsheba,BB,13.2167,-59.5167,NaN,27.34,74,40,10.29
4,4,Saint-François,FR,46.4154,3.9054,NaN,11.04,83,0,0.00
...,...,...,...,...,...,...,...,...,...,...
516,516,Esmeralda,CU,21.8525,-78.1169,NaN,23.57,94,83,2.01
517,517,Palmeira das Missões,BR,-27.8994,-53.3136,NaN,11.68,79,0,1.71
518,518,Betanzos,ES,43.2804,-8.2147,NaN,15.81,86,40,2.24
519,519,Tartagal,AR,-22.5164,-63.8013,NaN,15.34,44,0,1.37


In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = weather_df[["latitude","longitude"]]

#Add Heatmap layer to map
humidity = weather_df["humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
ideal_weather_df = weather_df[(weather_df['temperature'] >= 17) & (weather_df['temperature'] <= 40)]
ideal_weather_df = ideal_weather_df[ideal_weather_df['windspeed'] <= 10]
ideal_weather_df = ideal_weather_df[ideal_weather_df['cloudiness'] <= 10]
ideal_weather_df = ideal_weather_df[ideal_weather_df['humidity'] <= 70]
ideal_weather_df = ideal_weather_df
ideal_weather_df

,Unnamed: 0,city,country,latitude,longitude,date,temperature,humidity,cloudiness,windspeed
2,2,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,NaN,17.28,69,4,1.69
196,196,San Fernando del Valle de Catamarca,AR,-28.4696,-65.7852,NaN,18.76,57,0,1.75
282,282,San Miguel de Tucumán,AR,-26.8241,-65.2226,NaN,17.00,48,0,2.57
304,304,San Juan,AR,-31.5375,-68.5364,NaN,20.04,24,0,4.05
313,313,Tigre,AR,-34.4260,-58.5796,NaN,17.69,52,0,5.66
338,338,San Luis,AR,-33.2950,-66.3356,NaN,20.01,27,0,8.19
402,402,Villa Carlos Paz,AR,-31.4241,-64.4978,NaN,20.60,39,0,4.92
413,413,Camargo,MX,27.6667,-105.1667,NaN,17.99,32,0,2.01


Hotel Map

Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.

hotel_df = ideal_weather_df

hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,city,country,latitude,longitude,date,temperature,humidity,cloudiness,windspeed,Hotel Name
2,2,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,NaN,17.28,69,4,1.69,
196,196,San Fernando del Valle de Catamarca,AR,-28.4696,-65.7852,NaN,18.76,57,0,1.75,
282,282,San Miguel de Tucumán,AR,-26.8241,-65.2226,NaN,17.00,48,0,2.57,
304,304,San Juan,AR,-31.5375,-68.5364,NaN,20.04,24,0,4.05,
313,313,Tigre,AR,-34.4260,-58.5796,NaN,17.69,52,0,5.66,
338,338,San Luis,AR,-33.2950,-66.3356,NaN,20.01,27,0,8.19,
402,402,Villa Carlos Paz,AR,-31.4241,-64.4978,NaN,20.60,39,0,4.92,
413,413,Camargo,MX,27.6667,-105.1667,NaN,17.99,32,0,2.01,


In [6]:
# Set parameters to search for hotels with 5000 meters.

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['latitude']
        lng = row['longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,Unnamed: 0,city,country,latitude,longitude,date,temperature,humidity,cloudiness,windspeed,Hotel Name
2,2,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,NaN,17.28,69,4,1.69,NaN
196,196,San Fernando del Valle de Catamarca,AR,-28.4696,-65.7852,NaN,18.76,57,0,1.75,Amerian Catamarca Park Hotel
282,282,San Miguel de Tucumán,AR,-26.8241,-65.2226,NaN,17.00,48,0,2.57,Hotel Embajador
304,304,San Juan,AR,-31.5375,-68.5364,NaN,20.04,24,0,4.05,Hotel América
313,313,Tigre,AR,-34.4260,-58.5796,NaN,17.69,52,0,5.66,Hotel Agustín García
338,338,San Luis,AR,-33.2950,-66.3356,NaN,20.01,27,0,8.19,Vista Suites Spa & Golf
402,402,Villa Carlos Paz,AR,-31.4241,-64.4978,NaN,20.60,39,0,4.92,Portal Del Lago Hotel
413,413,Camargo,MX,27.6667,-105.1667,NaN,17.99,32,0,2.01,Hotel Santa Fe


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))